# [STARTER] Udaplay Project

## Part 01 - Offline RAG

In this part of the project, you'll build your VectorDB using Chroma.

The data is inside folder `project/starter/games`. Each file will become a document in the collection you'll create.
Example.:
```json
{
  "Name": "Gran Turismo",
  "Platform": "PlayStation 1",
  "Genre": "Racing",
  "Publisher": "Sony Computer Entertainment",
  "Description": "A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.",
  "YearOfRelease": 1997
}
```


### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
import os
import json
import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv

In [ ]:
# COMPLETED: Create a .env file with the following variables
# OPENAI_API_KEY="YOUR_KEY"
# CHROMA_OPENAI_API_KEY="YOUR_KEY"
# TAVILY_API_KEY="YOUR_KEY"

In [ ]:
# COMPLETED: Load environment variables
load_dotenv()
print("Environment variables loaded.")
print(f"OpenAI API Key: {os.getenv('OPENAI_API_KEY')[:10]}***")
print(f"Chrome OpenAI API Key: {os.getenv('CHROMA_OPENAI_API_KEY')[:10]}***")
print(f"Tavily API Key: {os.getenv('TAVILY_API_KEY')[:10]}***")

Environment variables loaded.
OpenAI API Key: sk-proj-Tu***
Chrome OpenAI API Key: sk-proj-Tu***
Tavily API Key: tvly-dev-g***


### VectorDB Instance

In [ ]:
# COMPLETED: Instantiate your ChromaDB Client
# Choose any path you want
chroma_client = chromadb.PersistentClient(path="chromadb")

### Collection

In [ ]:
# COMPLETED: Pick one embedding function
# If picking something different than openai, 
# make sure you use the same when loading it
embedding_fn = embedding_functions.OpenAIEmbeddingFunction()

In [ ]:
# COMPLETED: Create a collection
# Choose any name you want
collection = chroma_client.create_collection(
   name="udaplay",
   embedding_function=embedding_fn
)

### Add documents

In [13]:
# Make sure you have a directory "project/starter/games"
data_dir = "games"

for file_name in sorted(os.listdir(data_dir)):
    if not file_name.endswith(".json"):
        continue

    file_path = os.path.join(data_dir, file_name)
    with open(file_path, "r", encoding="utf-8") as f:
        game = json.load(f)

    # You can change what text you want to index
    content = f"[{game['Platform']}] {game['Name']} ({game['YearOfRelease']}) - {game['Description']}"

    # Use file name (like 001) as ID
    doc_id = os.path.splitext(file_name)[0]

    collection.add(
        ids=[doc_id],
        documents=[content],
        metadatas=[game]
    )

In [ ]:
# test by querying the collection
query = "What are racing games on PlayStation 5?"
results = collection.query(
    query_texts=[query],
    n_results=5
)

for doc, distance in zip(results['documents'][0], results['distances'][0]):
    print(f"Similarity: {1-distance:.3f}, Content: {doc[:100]}...")


Similarity: 0.718, Content: [PlayStation 3] Gran Turismo 5 (2010) - A comprehensive racing simulator featuring a vast selection ...
Similarity: 0.687, Content: [PlayStation 1] Gran Turismo (1997) - A realistic racing simulator featuring a wide array of cars an...
Similarity: 0.591, Content: [Nintendo Switch] Mario Kart 8 Deluxe (2017) - An enhanced version of Mario Kart 8, featuring new ch...
Similarity: 0.580, Content: [PlayStation 5] Marvel's Spider-Man 2 (2023) - The sequel to the acclaimed Spider-Man game, featurin...
Similarity: 0.554, Content: [PlayStation 4] Marvel's Spider-Man (2018) - An open-world superhero game that lets players swing th...
